In [1]:
import numpy
import dateutil.parser
import codecs

In [2]:
from elasticsearch import Elasticsearch

In [3]:
import os
import sys

# For Mac OS X and Linux
#spark_path = "/opt/spark"
# For Windows 7+
spark_path = "C:/opt/spark"

os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path

sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.9-src.zip")

from pyspark import SparkContext
from pyspark import SparkConf

sc = SparkContext("local", "test")

In [4]:
path = './Data//'
fileList = ['e1.txt','e2.txt','e3.txt','e4.txt','e5.txt']

In [5]:
sc

In [6]:
import glob

In [7]:
path = 'C:\\Users\\nisagarw\\Data\\poc_data_sorted2\\*'

In [8]:
files = glob.glob(path)  

In [9]:
files

['C:\\Users\\nisagarw\\Data\\poc_data_sorted2\\sats_events_2015-06-01-07-44-25_2015-06-02-07-44-25',
 'C:\\Users\\nisagarw\\Data\\poc_data_sorted2\\sats_events_2015-06-02-07-44-25_2015-06-03-07-44-25',
 'C:\\Users\\nisagarw\\Data\\poc_data_sorted2\\sats_events_2015-06-03-07-44-25_2015-06-04-07-44-25',
 'C:\\Users\\nisagarw\\Data\\poc_data_sorted2\\sats_events_2015-06-04-07-44-25_2015-06-05-07-44-25',
 'C:\\Users\\nisagarw\\Data\\poc_data_sorted2\\sats_events_2015-06-05-07-44-25_2015-06-06-07-44-25',
 'C:\\Users\\nisagarw\\Data\\poc_data_sorted2\\sats_events_2015-06-06-07-44-25_2015-06-07-07-44-25',
 'C:\\Users\\nisagarw\\Data\\poc_data_sorted2\\sats_events_2015-06-07-21-51-37_2015-06-08-21-51-37',
 'C:\\Users\\nisagarw\\Data\\poc_data_sorted2\\sats_events_2015-06-08-21-51-37_2015-06-09-21-51-37',
 'C:\\Users\\nisagarw\\Data\\poc_data_sorted2\\sats_events_2015-06-09-21-51-37_2015-06-10-21-51-37',
 'C:\\Users\\nisagarw\\Data\\poc_data_sorted2\\sats_events_2015-06-10-21-51-37_2015-06-11-2

In [10]:
import ujson
import socket, struct

def is_json(myjson):
        try:
            json_object = ujson.loads(myjson)
            p1 = (json_object["User"]["User"])
        except ValueError as e:
            return False
        except KeyError as e2:
            return False
        except:
            return False
        return True



# In[8]:

def safe_parse(raw_json):
    return raw_json.filter(lambda t : is_json(t)).map(lambda myjson : ujson.loads(myjson))
    pass

In [11]:
sc._conf.set('spark.executor.memory','32g').set('spark.driver.memory','32g').set('spark.driver.maxResultsSize','0')

In [12]:
k=0
for i in files:
    print("file:",k, " done!")
    k+=1
    content_rdd = sc.textFile(i)
    safe = safe_parse(content_rdd)
    rnd_logs = safe.filter(lambda x: x["User"]["Department"] == 'R&D' and len(x['SourceIP']) >1) 
    faulty_users  = rnd_logs.map(lambda x: ((x["User"]["User"],x['SourceIP']),1)) \
                    .reduceByKey(lambda x,y: x+y).map(lambda x: (x[0][0],(x[0][1],x[1])))   \
                    .reduceByKey(lambda x,y: x+y).collect()
    if(faulty_users):
        print(dict(faulty_users))

file: 0  done!


KeyboardInterrupt: 

In [ ]:
faulty_users  = rnd_logs.map(lambda x: ((x["User"]["User"],x['SourceIP'].split('.')[:3] ),1)).reduceByKey(lambda x,y: x+y) \
                    .map(lambda (x,y): (x[0],(x[1],y))).reduceByKey(lambda x,y: x+y).filter(lambda (x,y): len(y)>2).collect()

In [13]:
i = files[0]
from collections import Counter

In [14]:
import re

In [15]:
sc.textFile(i)
safe = safe_parse(content_rdd)

temp = safe.take(1)[0]['RawEventMessage']
safe.take(1)

[{'EventCollector': 'Imperva Inc.',
  'EventDescription': {'BindVariables': '',
   'DataStore': 'COE_PCRS_951/BANK_ORA3',
   'EventAffectedCount': '62',
   'EventApplicationName': 'Default Oracle Application',
   'IsSensitive': True,
   'Query': 'SELECT "COUNTRY", "ACCOUNT", "ZIP" FROM "BANK"."BANK_ORA3", "BANK"."BANK_LIN4"',
   'SensitiveDataDomains': [{'columns': [{'columnName': 'ACCOUNT',
       'schemaName': 'BANK',
       'tableName': 'BANK_ORA3'}],
     'dataDomainName': 'China_NationalID'},
    {'columns': [{'columnName': 'COUNTRY',
       'schemaName': 'BANK',
       'tableName': 'BANK_ORA3'}],
     'dataDomainName': 'DriverLicence_Canada'},
    {'columns': [{'columnName': 'ZIP',
       'schemaName': 'BANK',
       'tableName': 'BANK_ORA3'}],
     'dataDomainName': 'Passport_MachineReadable'}],
   'UserGroup': 'Default oracle group'},
  'EventMessageType': 'CEF',
  'EventUid': '729186814',
  'RawEventMessage': '<13>Apr 23 10:06:18 quickstart cloudera: CEF: 0|Imperva Inc.|Secure

In [16]:
def get_src_ip(x):
    temp=x['RawEventMessage']
    ip = re.search('src=(.+?) ', temp).group(1)
    if ip.endswith('|'):
        return ip[:-1]
    else:
        return ip
    


In [17]:
ip = '10.2.2.343|'
ip[:-1]
# if ip.endswith('|'):
#     print()


'10.2.2.343'

In [18]:
arr = safe.filter(lambda x: x['EventCollector'] == 'Imperva Inc.').take(1)
arr
# get_src_ip(safe.take(4)[3])

[{'EventCollector': 'Imperva Inc.',
  'EventDescription': {'BindVariables': '',
   'DataStore': 'COE_PCRS_951/BANK_ORA3',
   'EventAffectedCount': '62',
   'EventApplicationName': 'Default Oracle Application',
   'IsSensitive': True,
   'Query': 'SELECT "COUNTRY", "ACCOUNT", "ZIP" FROM "BANK"."BANK_ORA3", "BANK"."BANK_LIN4"',
   'SensitiveDataDomains': [{'columns': [{'columnName': 'ACCOUNT',
       'schemaName': 'BANK',
       'tableName': 'BANK_ORA3'}],
     'dataDomainName': 'China_NationalID'},
    {'columns': [{'columnName': 'COUNTRY',
       'schemaName': 'BANK',
       'tableName': 'BANK_ORA3'}],
     'dataDomainName': 'DriverLicence_Canada'},
    {'columns': [{'columnName': 'ZIP',
       'schemaName': 'BANK',
       'tableName': 'BANK_ORA3'}],
     'dataDomainName': 'Passport_MachineReadable'}],
   'UserGroup': 'Default oracle group'},
  'EventMessageType': 'CEF',
  'EventUid': '729186814',
  'RawEventMessage': '<13>Apr 23 10:06:18 quickstart cloudera: CEF: 0|Imperva Inc.|Secure

In [19]:
safe.map(lambda x : (  x['User']['Department'],x['User']['User']) ).distinct().countByKey()

defaultdict(int,
            {'Admin': 18,
             'Finance': 13,
             'HR': 14,
             'Legal': 17,
             'R&D': 14,
             'Sale': 22,
             'Support': 25})

In [20]:
safe.filter(lambda x: x['EventDescription']['IsSensitive'] == True ).map(lambda x: (x['EventDescription']['SensitiveDataDomains'][0]['dataDomainName'],1)).countByKey()

defaultdict(int,
            {'Account_Status': 937,
             'Age': 102,
             'China_NationalID': 6323,
             'Country': 1649,
             'CreditCard_AMEX': 115,
             'CreditCard_Visa': 667,
             'Date_AllFormats': 905,
             'Denmark_NationalID': 775,
             'DriverLicence_Canada': 1370,
             'DriversLicence_GBR': 794,
             'Finland_NationalID': 393,
             'Geocode_LatitudeLongtitude': 467,
             'GreatBritian_NINO': 738,
             'IBAN': 424,
             'India_NationalID': 2388,
             'Italy_FiscalCode': 56,
             'JobPosition': 154,
             'Passport_DEU_MR': 1044,
             'Passport_India': 200,
             'Passport_MachineReadable': 804,
             'PhoneNumber': 640,
             'Postcode': 170,
             'SouthAfrica_NationalID': 174})

In [21]:
def sens_domains(x):
    sens_dict = Counter({})
    if x['EventDescription']['IsSensitive'] == True :
         for i in x['EventDescription']['SensitiveDataDomains']:
                sens_dict[i['dataDomainName']]=1
    
    return sens_dict
        

In [22]:
sens_domains(temp)

TypeError: string indices must be integers

In [23]:
import time
from calendar import timegm

In [24]:
def combine(a,b):
    return tuple([i+j for i,j in zip(a,b)])

In [25]:
import dateutil.parser
def convert_hour(s):
    d = dateutil.parser.parse(s)
    return str(int(d.hour/6))

In [ ]:
k=0
for i in files:
    print("file:",k, " done!")
    k+=1
    content_rdd = sc.textFile(i)
    safe = safe_parse(content_rdd)
    rnd_logs = safe.filter(lambda x: x["User"]["Department"] == 'R&D' and len(x['SourceIP']) >1) \
                    .map(lambda x: (x["User"]["User"],Counter({x['SourceIP']:1})) )
    faulty_users  = rnd_logs.reduceByKey(lambda x,y: x+y).filter(lambda x: len(x[1])>1).collect()
    if(faulty_users):
        print(dict(faulty_users))

In [26]:
i = files[0]

In [26]:
text = 'gfgfdAAA1234ZZZuijjk'

m = re.search('AAA(.+?)ZZZu', text).group(1)
m

'1234'

In [114]:
k=0
for i in files:
    print("file:",k, " done!")
    k+=1
    content_rdd = sc.textFile(i)
    safe = safe_parse(content_rdd)
    rnd_logs = safe.filter(lambda x: x["User"]["Department"] == 'R&D' and len(x['SourceIP']) >1) \
                    .map(lambda x: (x["User"]["User"],(Counter({x['SourceIP']:1}), Counter({convert_hour(x['TimeOfEventUTC']):1}), sens_domains(x) )) )          
    faulty_users  = rnd_logs.reduceByKey(lambda x,y: combine(x,y)).collect()
    if(faulty_users):
        print(dict(faulty_users).keys())

file: 0  done!


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 523.0 failed 1 times, most recent failure: Lost task 0.0 in stage 523.0 (TID 1444, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\opt\spark\python\lib\pyspark.zip\pyspark\worker.py", line 111, in main
  File "C:\opt\spark\python\lib\pyspark.zip\pyspark\worker.py", line 106, in process
  File "C:/opt/spark/python\pyspark\rdd.py", line 2346, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:/opt/spark/python\pyspark\rdd.py", line 2346, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:/opt/spark/python\pyspark\rdd.py", line 317, in func
    return f(iterator)
  File "C:/opt/spark/python\pyspark\rdd.py", line 1776, in combineLocally
    merger.mergeValues(iterator)
  File "C:\opt\spark\python\

In [39]:
import json
file2 =  codecs.open("topics3.json",'w',encoding='utf8')

In [40]:
k=0
for i in files:
    print("file:",k, " done!")
    k+=1
    content_rdd = sc.textFile(i)
    safe = safe_parse(content_rdd)
    
    rnd_logs = safe.filter(lambda x: x["User"]["Department"] == 'Sale').filter(lambda x: x['EventCollector'] == 'Imperva Inc.').cache()
    cate_logs = rnd_logs.map(lambda x: (x["User"]["User"],(Counter({x['SourceIP']:1}), Counter({convert_hour(x['TimeOfEventUTC']):1}), sens_domains(x) )) )          
    cont_logs = rnd_logs.map(lambda x: (x["User"]["User"], ( int(x['EventDescription']['EventAffectedCount']), 0 if x['EventDescription']['IsSensitive'] == False else 1 )))
    cat_users  = cate_logs.reduceByKey(lambda x,y: combine(x,y))
    cont_users = cont_logs.reduceByKey(lambda x,y: combine(x,y))
#     u_feature_vec = cat_users.map(lambda x: (x[0],x[1][0])).collect()
    u_feature_vec = cat_users.leftOuterJoin(cont_users).collect()
    if(u_feature_vec):
#         print(dict(u_feature_vec))
        json.dump(dict(u_feature_vec), file2)
        file2.write('\n')

file: 0  done!
file: 1  done!
file: 2  done!
file: 3  done!
file: 4  done!
file: 5  done!
file: 6  done!
file: 7  done!
file: 8  done!
file: 9  done!
file: 10  done!
file: 11  done!
file: 12  done!
file: 13  done!
file: 14  done!
file: 15  done!
file: 16  done!
file: 17  done!
file: 18  done!
file: 19  done!
file: 20  done!
file: 21  done!
file: 22  done!
file: 23  done!
file: 24  done!
file: 25  done!
file: 26  done!
file: 27  done!
file: 28  done!
file: 29  done!


In [ ]:
a = (1,2,3)

In [ ]:
r={'df':1,'ty':77}
len(Counter(r).items())

In [ ]:
i = files[1]
line = open(i).readlines()[0]
content_rdd = sc.parallelize(line.replace('}{','}\n{').split('\n')).cache()
content_rdd.count()


In [ ]:
content_rdd

In [ ]:
safe = safe_parse(content_rdd)

In [ ]:
safe.map(lambda x: (x["User"]["Department"], x["User"]["User"])).distinct().countByKey()

In [ ]:
from geolite2 import geolite2

In [ ]:
rnd_logs = safe.filter(lambda x: x["User"]["Department"] == 'R&D' and len(x['SourceIP']) >1)

In [ ]:
names = rnd_logs.map(lambda x: x["User"]["User"]).distinct().collect()

In [ ]:
temp = rnd_logs.map(lambda x: (tuple(x['SourceIP'].split('.')[:3]),1) ).countByKey()

In [ ]:
temp

In [ ]:
a = {'Mendoza,  Austin D.': (('10', '77', '0'), 26), 'Cherry,  Wynne S.': (('10', '77', '0'), 20), 'Graves,  Kylan I.': (('10', '77', '0'), 28), 'Avila,  Tiger Y.': (('10', '77', '5'), 29), 'Bonner,  Wing P.': (('10', '77', '4'), 1), 'Greer,  Mannix X.': (('10', '77', '1'), 1), 'Wells,  Mary O.': (('10', '77', '3'), 85), 'Washington,  Russell X.': (('10', '77', '0'), 6), 'Pena,  Burton Y.': (('10', '77', '3'), 170)}
b = {'Blackburn,  Quyn K.': (('10', '77', '2'), 350), 'Golden,  McKenzie W.': (('10', '77', '4'), 45), 'Bonner,  Wing P.': (('10', '77', '4'), 56), 'Greer,  Mannix X.': (('10', '77', '1'), 325), 'Mendoza,  Austin D.': (('10', '77', '0'), 7), 'Cherry,  Wynne S.': (('10', '77', '0'), 15), 'Zamora,  Teegan W.': (('10', '77', '1'), 327), 'Thornton,  Guinevere L.': (('10', '77', '2'), 65), 'Chandler,  Madeline G.': (('10', '77', '1'), 293), 'Washington,  Russell X.': (('10', '77', '0'), 31), 'Wells,  Mary O.': (('10', '77', '3'), 5), 'Crane,  Shelley K.': (('10', '77', '1'), 480)}